In [1]:
import sympy as sm
from sympy import diff, Symbol, cos
from scipy.integrate import solve_ivp,OdeSolver,odeint
import numpy as np
from sympy.physics.mechanics import dynamicsymbols
import math
import matplotlib.pyplot as plt

In [2]:
from sympy.physics.vector import init_vprinting
init_vprinting(use_latex="mathjax", pretty_print=False)

In [3]:
m1, m2, l1, l2, gty ,tau1, tau2, c1, c2, L, I1, I2 = sm.symbols('m1 m2 l1 l2 g tau1 tau2 c1 c2 L I1 I2', real =True)
t = Symbol('t')
theta1, theta2, phi1, phi2 =  dynamicsymbols('theta1 theta2 phi1 phi2')

theta1d = dynamicsymbols('theta1',1)
theta2d = dynamicsymbols('theta2',1)
phi1d = dynamicsymbols('phi1',1)
phi2d = dynamicsymbols('phi2',1)
theta1dd = dynamicsymbols('theta1',2)
theta2dd = dynamicsymbols('theta2',2)
phi1dd = dynamicsymbols('phi1',2)
phi2dd = dynamicsymbols('phi2',2)
gty, m1, m2, l1, l2, theta1, theta2, phi1, phi2, theta1d, theta2d, phi1d,phi2d, theta1dd, theta2dd, phi1dd,phi2dd, tau1, tau2,I1, I2

(g, m1, m2, l1, l2, theta1, theta2, phi1, phi2, theta1', theta2', phi1', phi2', theta1'', theta2'', phi1'', phi2'', tau1, tau2, I1, I2)

In [4]:
x1 = l1*sm.cos(theta1)*sm.sin(phi1)
y1 = l1*sm.sin(theta1)*sm.sin(phi1)
z1 = l1*sm.cos(phi1)

x2 = x1 + l2*sm.cos(theta2)*sm.sin(phi2)
y2 = y1 + l2*sm.sin(theta2)*sm.sin(phi2)
z2 = z1 + l2*cos(phi2 )

x1_dot = sm.diff(x1,t)
x2_dot = sm.diff(x2,t)
y1_dot = sm.diff(y1,t)
y2_dot = sm.diff(y2,t)
z1_dot = sm.diff(z1,t)
z2_dot = sm.diff(z2,t)

x1_dot , x2_dot, y1_dot , y2_dot, z1_dot, z2_dot

(-l1*sin(phi1)*sin(theta1)*theta1' + l1*cos(phi1)*cos(theta1)*phi1', -l1*sin(phi1)*sin(theta1)*theta1' + l1*cos(phi1)*cos(theta1)*phi1' - l2*sin(phi2)*sin(theta2)*theta2' + l2*cos(phi2)*cos(theta2)*phi2', l1*sin(phi1)*cos(theta1)*theta1' + l1*sin(theta1)*cos(phi1)*phi1', l1*sin(phi1)*cos(theta1)*theta1' + l1*sin(theta1)*cos(phi1)*phi1' + l2*sin(phi2)*cos(theta2)*theta2' + l2*sin(theta2)*cos(phi2)*phi2', -l1*sin(phi1)*phi1', -l1*sin(phi1)*phi1' - l2*sin(phi2)*phi2')

# dynamic modeling on manipulator without constraint using lagrange equation

In [6]:
# kinetic energy
ke = (sm.Rational(1/2))*m1*(x1_dot**2)
ke += (sm.Rational(1/2))*m1*(y1_dot**2)
ke += (sm.Rational(1/2))*m1*(z1_dot**2)
ke += (sm.Rational(1/2))*m2*(x2_dot**2)
ke += (sm.Rational(1/2))*m2*(y2_dot**2)
ke += (sm.Rational(1/2))*m2*(z2_dot**2)
sm.simplify(ke)

l1**2*m1*sin(phi1)**2*theta1'**2/2 + l1**2*m1*phi1'**2/2 + l1**2*m2*sin(phi1)**2*theta1'**2/2 + l1**2*m2*phi1'**2/2 - l1*l2*m2*sin(theta1 - theta2)*sin(phi1)*cos(phi2)*phi2'*theta1' + l1*l2*m2*sin(theta1 - theta2)*sin(phi2)*cos(phi1)*phi1'*theta2' + l1*l2*m2*sin(phi1)*sin(phi2)*cos(theta1 - theta2)*theta1'*theta2' + l1*l2*m2*sin(phi1)*sin(phi2)*phi1'*phi2' + l1*l2*m2*cos(theta1 - theta2)*cos(phi1)*cos(phi2)*phi1'*phi2' + l2**2*m2*sin(phi2)**2*theta2'**2/2 + l2**2*m2*phi2'**2/2

In [7]:
# potential energy
pe = m1*gty*y1
pe += m2*gty*y2
sm.expand(pe)

g*l1*m1*sin(phi1)*sin(theta1) + g*l1*m2*sin(phi1)*sin(theta1) + g*l2*m2*sin(phi2)*sin(theta2)

In [11]:
# the lagrangian equation
Ls = ke - pe

In [28]:
LE1 = (sm.diff(sm.diff(Ls,theta1d),t) - sm.diff(Ls,theta1)).expand()
LE2 = (sm.diff(sm.diff(Ls,theta1d),t) - sm.diff(Ls,theta2)).expand()
LE3 = (sm.diff(sm.diff(Ls,phi1d),t) - sm.diff(Ls,phi1)).expand()
LE4 = (sm.diff(sm.diff(Ls,phi2d),t) - sm.diff(Ls,phi2)).expand()

In [29]:
#inertia term
it11 = LE1.coeff(theta1dd) * theta1dd
it12 = LE2.coeff(theta1dd) * theta1dd
it13 = LE3.coeff(theta1dd) * theta1dd
it14 = LE4.coeff(theta1dd) * theta1dd

it21 = LE1.coeff(theta2dd) * theta2dd
it22 = LE2.coeff(theta2dd) * theta2dd
it23 = LE3.coeff(theta2dd) * theta2dd
it24 = LE4.coeff(theta2dd) * theta2dd

it31 = LE1.coeff(phi1dd) * phi1dd
it32 = LE2.coeff(phi1dd) * phi1dd
it33 = LE3.coeff(phi1dd) * phi1dd
it34 = LE4.coeff(phi1dd) * phi1dd

it41 = LE1.coeff(phi2dd) * phi2dd
it42 = LE2.coeff(phi2dd) * phi2dd
it43 = LE3.coeff(phi2dd) * phi2dd
it44 = LE4.coeff(phi2dd) * phi2dd


m11 = LE1.coeff(theta1dd)
m12 = LE2.coeff(theta1dd)
m13 = LE3.coeff(theta1dd)
m14 = LE4.coeff(theta1dd)

m21 = LE1.coeff(theta2dd)
m22 = LE2.coeff(theta2dd)
m23 = LE3.coeff(theta2dd)
m24 = LE4.coeff(theta2dd)

m31 = LE1.coeff(phi1dd)
m32 = LE2.coeff(phi1dd)
m33 = LE3.coeff(phi1dd)
m34 = LE4.coeff(phi1dd)

m41 = LE1.coeff(phi2dd)
m42 = LE2.coeff(phi2dd)
m43 = LE3.coeff(phi2dd)
m44 = LE4.coeff(phi2dd)

M = sm.Matrix([[m11, m21, m31, m41], [m12, m22, m32, m42], [m13, m23, m33, m43], [m14, m24, m34, m44]])
M

Matrix([
[l1**2*m1*sin(phi1)**2*sin(theta1)**2 + l1**2*m1*sin(phi1)**2*cos(theta1)**2 + l1**2*m2*sin(phi1)**2*sin(theta1)**2 + l1**2*m2*sin(phi1)**2*cos(theta1)**2, l1*l2*m2*sin(phi1)*sin(phi2)*sin(theta1)*sin(theta2) + l1*l2*m2*sin(phi1)*sin(phi2)*cos(theta1)*cos(theta2),                                                                                                                                                                                                         0,                               -l1*l2*m2*sin(phi1)*sin(theta1)*cos(phi2)*cos(theta2) + l1*l2*m2*sin(phi1)*sin(theta2)*cos(phi2)*cos(theta1)],
[l1**2*m1*sin(phi1)**2*sin(theta1)**2 + l1**2*m1*sin(phi1)**2*cos(theta1)**2 + l1**2*m2*sin(phi1)**2*sin(theta1)**2 + l1**2*m2*sin(phi1)**2*cos(theta1)**2, l1*l2*m2*sin(phi1)*sin(phi2)*sin(theta1)*sin(theta2) + l1*l2*m2*sin(phi1)*sin(phi2)*cos(theta1)*cos(theta2),                                                                                                                      